# Make Volumetric Graphs of Line Mode - Second Try

In [14]:
colors_base = {
"00":"c64_black",  
"01":"c64_white",  
"02":"c64_red",    
"03":"c64_cyan",   
"04":"c64_purple", 
"05":"c64_green",  
"06":"c64_blue",   
"07":"c64_yellow", 
"08":"c64_orange", 
"09":"c64_brown",  
"0a":"c64_ltred",  
"0b":"c64_gray1",  
"0c":"c64_lightgray",  
"0d":"c64_ltgreen",
"0e":"c64_ltblue", 
"0f":"c64_gray3",  
}
colors = [v for k,v in colors_base.items()]
    
c64_to_rgb = {
"c64_black": "#ffffff",     
"c64_white": "#ffffff",     
"c64_red": "#880000",       
"c64_cyan":  "#aaffee",     
"c64_purple": "#cc44cc",    
"c64_green": "#00cc55",     
#"c64_blue":  "#0000aa",     
"c64_blue":  "#0088ff",     
"c64_yellow":  "#eeee77",   
"c64_orange":  "#dd8855",   
"c64_brown": "#664400",     
"c64_ltred": "#ff7777",  
"c64_gray1":  "#333333",    
"c64_ltgreen": "#aaff66",
"c64_ltblue":  "#0088ff",
"c64_lightgrey": "#bbbbbb", 
"c64_lightgray": "#bbbbbb", 
"c64_gray3": "#bbbbbb",                             
}


Methods for culling the pixel data to just the volume that's occupied.

In [10]:
def firstNonZero(l):
    for i,e in enumerate(l):
        if e > 0:
            return i
    return i

def getLeftMostPixelForPattern(pattern):
    return min([firstNonZero(r) for scr in pattern for r in scr])

def getRightMostPixelForPattern(pattern):
    return len(pattern[0][0]) - min([firstNonZero(list(reversed(r)))
                     for scr in pattern for r in scr])

def rotatePatternToCols(pattern):
    pattern_cols = []
    for scr in pattern:
        col_arr = [[0 for i in range(len(scr))] for j in range(len(scr[0]))]
        for i,r in enumerate(scr):
            for j,v in enumerate(r):
                col_arr[j][i] = v
        pattern_cols += [col_arr]
    return pattern_cols

def cullPattern(pattern):
    first_left = getLeftMostPixelForPattern(pattern)
    last_right = getRightMostPixelForPattern(pattern)
    width = last_right - first_left

    pattern_as_cols = rotatePatternToCols(pattern)
    first_top = getLeftMostPixelForPattern(pattern_as_cols)
    last_bottom = getRightMostPixelForPattern(pattern_as_cols)
    height = last_bottom - first_top
    
    culled = [[r[first_left:last_right] 
              for r in scr[first_top:last_bottom]]
              for scr in pattern]
    return culled

#cullPattern(patterns[0])
#patterns[0]

## Create the trace file and create images from it

Create the trace file

In [17]:
import os
log_file = "PsychedeliaLineModePatternsPixels.txt"
open('linemodetrace.txt','w').write(f"""log on
logname "{log_file}"
tr exec 1358
command 1 \"mem d800 dbff\"
tr exec 1362
command 2 \"mem d800 dbff\"
""")


133

In [28]:
os.chdir('../psychedelia')
! make psychedelia.prg
os.chdir('../notebooks')
! rm PsychedeliaLineModePatternsPixels.txt
! ~/Dev/vice-emu-code/vice/src/x64sc -directory ~/Dev/vice-emu-code/vice/data/ -moncommands linemodetrace.txt ../psychedelia/bin/psychedelia.prg 

64tass -Wall --cbm-prg -Wno-implied-reg -o bin/psychedelia.prg -L bin/list-co1.txt -l bin/labels.txt src/c64/psychedelia.asm
64tass Turbo Assembler Macro V1.56.2625?
64TASS comes with ABSOLUTELY NO WARRANTY; This is free software, and you
are welcome to redistribute it under certain conditions; See LICENSE!

Assembling file:   src/c64/psychedelia.asm
Assembling file:   src/c64/constants.asm
Assembling file:   src/c64/presets.asm
Assembling file:   src/c64/burst_generators.asm
Assembling file:   src/c64/sequencer_data.asm
Assembling file:   src/c64/custom_patterns.asm
Error messages:    None
Warning messages:  None
Passes:            3
Memory range:      $0801-$3000   $2800
echo "4b67db818f0203829595d58a5f613d37  bin/psychedelia.prg" | md5sum -c
bin/psychedelia.prg: OK
archdep_create_user_state_dir(): state dir = /home/robert/.local/state/vice
Detecting ISA HardSID boards.
Could not open '/dev/port'.
Cannot get permission to access $300.
Detecting PCI HardSID boards.
No PCI HardSID boar

reSID: MOS8580, filter on, sampling rate 48000Hz - resampling, pass to 21600Hz
Sync reset
AUTOSTART: Loading program 'PSYCHEDELIA'
AUTOSTART: Entered ROM at $e5cd
AUTOSTART: Searching for ...
AUTOSTART: Loading
AUTOSTART: Entered ROM at $ea21
AUTOSTART: Ready
AUTOSTART: Turning Warp mode off.
AUTOSTART: Starting program.
AUTOSTART: Restoring FileSystemDevice8 to 1.
AUTOSTART: Restoring FSDevice8ConvertP00 to 1.
AUTOSTART: Restoring FSDeviceLongNames to 0.
AUTOSTART: Done.
Sync reset
Initiating render thread 0 shutdown
Joining render thread 0 ...
Joined render thread 0.

Resources with non default values:
Directory="/home/robert/Dev/vice-emu-code/vice/data/"
SaveResourcesOnExit=1
Window0Height=650
Window0Width=720
Window0Xpos=906
Window0Ypos=213
SoundBufferSize=100
SoundVolume=76
KeymapIndex=1
KeySet1North=65362
KeySet1East=65363
KeySet1South=65364
KeySet1West=65361
KeySet1Fire=65507
JoyDevice1=3
JoyDevice2=2
MonitorLogFileName="PsychedeliaLineModePatternsPixels.txt"
MonitorLogEnabled=1

Read in the ram history.

In [29]:
input_file = open(log_file,'r')

ram_history = []
lines = input_file.readlines()
c = 0
for i in range(7, len(lines), 67):
    raw_ram = [l[9:60].split() for l in lines[i:i+64]]
    raw_ram = [int(v,16) & 0x0f for l in raw_ram for v in l]
    color_ram = list(reversed([raw_ram[v:v+40] for v in range(0, len(raw_ram), 40)]))[2:]
    ram_history += [color_ram]
    c += 1


Slice up the ram history into the different patterns.

In [30]:
flatten = lambda l: [e for sublist in l for e in sublist]

pattern_breaks = []
for i,h in enumerate(ram_history):
    num_pixels = sum(flatten(h))
    if num_pixels > 1:
        continue
    pattern_breaks += [i]

patterns = []
pp = 0
for p in pattern_breaks:
    patterns += [ram_history[pp:p]]
    pp = p
#patterns += [ram_history[pp:p]]

[len(p) for p in patterns]

[55, 2, 1, 1, 90, 30, 45]

 Finally, write out our visualisation of each pattern evolution.

In [31]:
from PIL import Image, ImageColor

bit_array = ([['0' for i in range(8)]] 
            + [['0']+['1' for i in range(6)]+['0'] for j in range(6)] 
            + [['0' for i in range(8)]])

def paintChar(origin, pixels, cv):
    c, r = origin
    x_o, y_o = c*8, r*8
    for y, l in enumerate(bit_array):
        for x,bit in enumerate(l):
            if bit == "0":
                continue
            color = c64_to_rgb[colors[cv]]
            pixels[x_o+x, y_o+y] = ImageColor.getrgb(color)


for i,pattern in enumerate(patterns):
    cull_pattern = reversed(cullPattern(pattern))
    for j, scr in enumerate(cull_pattern):
        cols=len(scr[0])
        rows=len(scr)
        bits=8
        img = Image.new( 'RGB', (cols*bits,rows*bits), "black")
        pixels = img.load()
        for y, l in enumerate(scr):
            for x, v in enumerate(l):
                paintChar((x,y), pixels, v)
        img = img.resize((img.width * 4, img.height * 4), Image.NEAREST)
        img.save(f"linemode/pixels/pixel_pattern{i}_{j}.png")

